In [1]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import t

def four_param_logistic(conc, A, B, C, D):
    return D + (A - D) / (1 + (conc / C)**B)

data_folder = os.getcwd()

results = []

for filename in os.listdir(data_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)

        if not set(["X", "Y", "Condition"]).issubset(df.columns):
            continue

        parts = filename.replace(".csv", "").split("_")
        compound_name = parts[1] if len(parts) > 1 else filename.replace(".csv", "")

        for condition in df["Condition"].unique():
            subset = df[df["Condition"] == condition]
            x = subset["X"].values
            y = subset["Y"].values

            if len(x) < 11:
                continue

            try:
                A_init = max(y)
                D_init = min(y)
                C_init = np.median(x)
                B_init = 1.0
                p0 = [A_init, B_init, C_init, D_init]

                popt, pcov = curve_fit(
                    four_param_logistic, x, y, p0=p0,
                    bounds=(0, np.inf), maxfev=10000
                )
                A, B, C, D = popt

                dof = max(0, len(x) - len(popt))
                tval = t.ppf(0.975, dof)
                perr = np.sqrt(np.diag(pcov))
                ci = perr * tval

                results.append({
                    "compound": compound_name,
                    "condition": condition,
                    "Top (A)": round(A, 4),
                    "Slope (B)": round(B, 4),
                    "EC50 (C, µM)": round(C, 6),
                    "Bottom (D)": round(D, 4),
                    "C CI lower (µM)": round(C - ci[2], 6),
                    "C CI upper (µM)": round(C + ci[2], 6)
                })

            except Exception as e:
                results.append({
                    "compound": compound_name,
                    "condition": condition,
                    "Top (A)": None,
                    "Slope (B)": None,
                    "EC50 (C, µM)": None,
                    "Bottom (D)": None,
                    "C CI lower (µM)": None,
                    "C CI upper (µM)": None,
                    "error": str(e)
                })

results_df = pd.DataFrame(results)
results_df.to_csv("logistic_fit_results.csv", index=False)

print("Saved results to 'logistic_fit_results.csv'")


Saved results to 'logistic_fit_results.csv'
